# Worksheet 08

Name: Wanchen Hong  
UID: U72264072

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[6.467952670636409, 5.563366304408252, 5.091455916755247, 3.566864947265464, 3.3481681026602335, 3.879527384294929, 5.864279824522972, 5.373143220840556, 3.64355553492833, 5.782413452639796]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[7.055197836081992, 7.878057710416484, 6.8372291481310405, 7.809974973823113, 8.351251758547985, 6.818770432193924, 8.429256443546912, 8.070478401625305, 9.260808834942011, 8.119251555891127]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[5.782413452639796, 8.119251555891127, 3.64355553492833, 9.260808834942011, 5.373143220840556, 5.864279824522972, 8.070478401625305, 3.879527384294929, 8.429256443546912, 3.3481681026602335]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

P(S1) = P(S2) = 0.5

Mean1 = 5, Mean2 = 8

Var1 = Var2 = 1

Size1 = Size2 = 10

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [8]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

c:\Files\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Files\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


[5.782413452639796, 3.64355553492833, 5.373143220840556, 5.864279824522972, 3.879527384294929, 3.3481681026602335]
[8.119251555891127, 9.260808834942011, 8.070478401625305, 8.429256443546912]
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 4.648514586647803,  mean_2 = 8.469948809001338
var_1 = 1.096847233474981,  var_2 = 0.22742015473634505


The means and variance for dataset 1 are bit far from the true values, but var 2 is very far. The probability of choosing is also not very far from the true values

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [9]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  5.782413452639796
probability of observing that point if it came from cluster 0 =  0.2131563988378968
probability of observing that point if it came from cluster 1 =  8.294751504522155e-31
point =  8.119251555891127
probability of observing that point if it came from cluster 0 =  0.0024351974869489226
probability of observing that point if it came from cluster 1 =  0.5342086903677319
point =  3.64355553492833
probability of observing that point if it came from cluster 0 =  0.23904278299157236
probability of observing that point if it came from cluster 1 =  2.7759489123942705e-98
point =  9.260808834942011
probability of observing that point if it came from cluster 0 =  5.2610813050442035e-05
probability of observing that point if it came from cluster 1 =  0.004150261105095402
point =  5.373143220840556
probability of observing that point if it came from cluster 0 =  0.2924078937240837
probability of observing that point if it came from cluster 1 =  9.536671451208836e-41
point 

Looks just like the truth, the ones close to 5 are more likely to belong to that cluster and similarly for the other one.

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [14]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum(prob_s0_x[i] * (data[i] - mean[0])**2 for i in range(len(data))) / sum(prob_s0_x), 
       sum(prob_s1_x[i] * (data[i] - mean[1])**2 for i in range(len(data)))/sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 4.673472028405649,  mean_2 = 8.467950776492012
var_1 = 1.1921865323693985,  var_2 = 0.22595773385386086


This is the same as what I got from K means

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [18]:
prob_s0_x_new = [norm.pdf(data[i], mean[0], var[0]**0.5) * prob_s[0] / 
                     (norm.pdf(data[i], mean[0], var[0]**0.5) * prob_s[0] + 
                      norm.pdf(data[i], mean[1], var[1]**0.5) * prob_s[1]) for i in range(len(data))]

prob_s1_x_new = [norm.pdf(data[i], mean[1], var[1]**0.5) * prob_s[1] / 
                     (norm.pdf(data[i], mean[0], var[0]**0.5) * prob_s[0] + 
                      norm.pdf(data[i], mean[1], var[1]**0.5) * prob_s[1]) for i in range(len(data))]
print("old:",prob_s0_x, "\nnew:" ,prob_s0_x_new)
print("old:",prob_s1_x, "\nnew:" ,prob_s1_x_new)

old: [1.0, 0.00679133256691503, 1.0, 0.018659945780779237, 1.0, 1.0, 0.011075705146442214, 1.0, 0.0008305904760044631, 1.0] 
new: [0.9999996992824961, 0.005842434629902242, 1.0, 0.0003854285804361495, 0.9999999988254922, 0.999999151649614, 0.007273296620835514, 1.0, 0.001763296269263804, 1.0]
old: [2.594261474905797e-30, 0.993208667433085, 7.741846815463542e-98, 0.9813400542192208, 2.1742849984315963e-40, 1.853964933989907e-28, 0.9889242948535577, 1.389435352244687e-88, 0.9991694095239956, 4.721952362420189e-110] 
new: [3.007175038469606e-07, 0.9941575653700978, 1.0256666711929049e-22, 0.9996145714195639, 1.174507841978209e-09, 8.483503860122132e-07, 0.9927267033791645, 1.1672643804814352e-20, 0.9982367037307361, 2.0647694703329448e-25]


The probabilities changed slightly, but are still essntially either 0 or 1.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [19]:
assignments = []

for i in range(len(data)):
    if prob_s0_x[i] > prob_s1_x[i]:
       assignments.append(0)
    else:
        assignments.append(1)
for i in range(len(data)):
    print("point = ", data[i], "cluster = ", assignments[i])

point =  5.782413452639796 cluster =  0
point =  8.119251555891127 cluster =  1
point =  3.64355553492833 cluster =  0
point =  9.260808834942011 cluster =  1
point =  5.373143220840556 cluster =  0
point =  5.864279824522972 cluster =  0
point =  8.070478401625305 cluster =  1
point =  3.879527384294929 cluster =  0
point =  8.429256443546912 cluster =  1
point =  3.3481681026602335 cluster =  0
